In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as pyplot
import os
import re
import math
from enum import Enum

In [ ]:
class Filter(Enum):
  RECTANGLE = cv2.MORPH_RECT
  CROSS = cv2.MORPH_CROSS
  ELLIPSE = cv2.MORPH_ELLIPSE

In [ ]:
ASSETS_FOLDER = "assets/"
OUTPUT_FOLDER = "output/"
SHAPE = (5, 5)

In [ ]:
def display_image(image, title):
  imgplot = pyplot.imshow(image, cmap="gray")
  pyplot.axis('off')
  pyplot.title(title)
  pyplot.show()

In [ ]:
def display_images(images, titles):
  fig, ax = pyplot.subplots(1, len(images))
  fig.set_figheight(15)
  fig.set_figwidth(15)
  fig.tight_layout()
  
  for n in range(len(images)):
    ax[n].set_title(titles[n], fontsize=16)
    ax[n].imshow(images[n])
    ax[n].set_axis_off()

In [ ]:
def fix_image(image):
  image[image > 1] = 1
  image[image <= 0] = 0
  return image

In [ ]:
def load_image(filename):
  image = cv2.imread(ASSETS_FOLDER + filename, -1)
  max_value = np.iinfo(image.dtype).max

  if image is not None:
    image = image.astype(np.float32)
    image = image / max_value
    image = fix_image(image)

    return image

In [ ]:
def get_filter(shape, type):
  return cv2.getStructuringElement(type, shape)

In [ ]:
def erode(image, filter):
  return cv2.erode(image, filter, iterations=1)

In [ ]:
def dilate(image, filter):
  return cv2.dilate(image, filter, iterations=1)

In [ ]:
for filter in Filter:
  image = load_image("j.png")
  images = [
    image,
    erode(image, get_filter(SHAPE, filter.value)),
    dilate(image, get_filter(SHAPE, filter.value))
  ]
  titles = [
    "ORIGINAL", "ERODED WITH " + filter.name + " FILTER", "DILATED WITH " + filter.name + " FILTER"
  ]

  display_images(images, titles)